In [2]:
import os, sys
import shutil
import json
import logging
from pathlib import Path
import multiprocessing
import argparse
import functools
from datetime import datetime
from typing import *

import numpy as np
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, Subset
from torch.utils.data.dataloader import DataLoader
import torch.nn.functional as F
from einops import rearrange
from torch.distributions import Normal, kl_divergence

import pytorch_lightning as pl
from pytorch_lightning.strategies.ddp import DDPStrategy

from transformers import BertConfig

from foldingdiff import datasets
from foldingdiff import modelling
from foldingdiff import losses
from foldingdiff import beta_schedules
from foldingdiff import plotting
from foldingdiff import utils
from foldingdiff import custom_metrics as cm

assert torch.cuda.is_available(), "Requires CUDA to train"
# reproducibility
torch.manual_seed(6489)
# torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False

# Define some typing literals
ANGLES_DEFINITIONS = Literal[
    "canonical", "canonical-full-angles", "canonical-minimal-angles", "cart-coords"
]

/home/sh2748/.conda/envs/foldingdiff/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def modulo_with_wrapped_range(
    vals, range_min: float = -np.pi, range_max: float = np.pi
):
    """
    Modulo with wrapped range -- capable of handing a range with a negative min

    modulo_with_wrapped_range(3, -2, 2)
    -1
    """
    assert range_min <= 0.0
    assert range_min < range_max

    # Modulo after we shift values
    top_end = range_max - range_min
    # Shift the values to be in the range [0, top_end)
    vals_shifted = vals - range_min
    # Perform modulo
    vals_shifted_mod = vals_shifted % top_end
    # Shift back down
    retval = vals_shifted_mod + range_min

    # Checks
    # print("Mod return", vals, " --> ", retval)
    # if isinstance(retval, torch.Tensor):
    #     notnan_idx = ~torch.isnan(retval)
    #     assert torch.all(retval[notnan_idx] >= range_min)
    #     assert torch.all(retval[notnan_idx] < range_max)
    # else:
    #     assert (
    #         np.nanmin(retval) >= range_min
    #     ), f"Illegal value: {np.nanmin(retval)} < {range_min}"
    #     assert (
    #         np.nanmax(retval) <= range_max
    #     ), f"Illegal value: {np.nanmax(retval)} > {range_max}"
    return retval

In [4]:
# Define a PyTorch Dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe=None, data_path=None):
        if data_path is not None:
            print(f"Initializing from a data path: {data_path}")
            self.data = torch.load(data_path)
        elif data_path is None and dataframe is not None:
            print("Initializing from a dataframe.")
            self.data = torch.stack(dataframe['angles'].tolist())
        else:
            raise Exception("Dataframe and data path are both None!")
        
         
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [5]:
# Initialize the dataset with the DataFrame
dataset = MyDataset(data_path="protein_angles_100sanity_128.pt")
dataloader = DataLoader(
    dataset,
    batch_size=8,       # Number of samples per batch
    shuffle=True,        # Shuffle the data at every epoch
    num_workers=1        # Number of subprocesses to use for data loading
)

Initializing from a data path: protein_angles_100sanity_128.pt


/tmp/tmp.iD1eevmCgx/ipykernel_1484158/2395865528.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data = torch.load(data_path)


In [6]:
from IntegralFlowMatching.integral_flow_matching import IntegralFlowMatcher
IFM = IntegralFlowMatcher(sigma=0.1, same_time=True, time_interp=True, noise_on_x0=True, noise_on_x1=True)

Noise is not varying.


In [7]:
batch = next(iter(dataloader))
x0 = torch.randn_like(batch)

In [8]:
t, xt, mu_t = IFM.sample_conditional_flow(rearrange(x0, 'b s d -> b (s d)').to("cuda"), rearrange(batch, 'b s d -> b (s d)').to("cuda"), 8,  device = "cuda")
xt = rearrange(xt, 'b t (s d) -> b t s d', s=x0.shape[1])
mu_t = rearrange(mu_t, 'b t (s d) -> b t s d', s=x0.shape[1])

print(f"Shapes:{xt.shape}, {mu_t.shape}")
print(f"Any out of range values in xt: {((xt < -np.pi) | (xt > np.pi)).any().item()}")
print(f"Any out of range values in mu_t: {((mu_t < -np.pi) | (mu_t > np.pi)).any().item()}")
xt = modulo_with_wrapped_range(xt)
mu_t = modulo_with_wrapped_range(mu_t)
print("After transformation:")
print(f"Shapes:{xt.shape}, {mu_t.shape}")
print(f"Any out of range values in xt: {((xt < -np.pi) | (xt > np.pi)).any().item()}")
print(f"Any out of range values in mu_t: {((mu_t < -np.pi) | (mu_t > np.pi)).any().item()}")

Shapes:torch.Size([8, 8, 128, 6]), torch.Size([8, 8, 128, 6])
Any out of range values in xt: True
Any out of range values in mu_t: True
After transformation:
Shapes:torch.Size([8, 8, 128, 6]), torch.Size([8, 8, 128, 6])
Any out of range values in xt: False
Any out of range values in mu_t: False


In [9]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.1):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=dropout_prob)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

import torch
from torch import nn
from torch.distributions import Normal
from ipdb import set_trace

# Copied from DanielFLevine/ifm/utils/modules.py

class ResidualBlock(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_prob=0.1):
        super(ResidualBlock, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.norm = nn.LayerNorm(output_dim, elementwise_affine=False)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        identity = x
        out = self.linear(x)
        out = self.norm(out)
        out = self.relu(out)
        out = self.dropout(out)
        # out += identity  # Skip connection
        return out

class MLPLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_prob=0.1):
        super(MLPLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.norm = nn.LayerNorm(output_dim, elementwise_affine=False)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        identity = x
        out = self.linear(x)
        out = self.norm(out)
        out = self.relu(out)
        out = self.dropout(out)
        return out

class MidFC(nn.Module):
    def __init__(self, dim, num_layers, dropout_prob=0.1):
        super(MidFC, self).__init__()
        self.layers = nn.Sequential(
            *[MLPLayer(dim, dim) for _ in range(num_layers)]
        )
        
    def forward(self, x):
        return self.layers(x)


class CustomDecoder(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim, num_blocks=1):
        super(CustomDecoder, self).__init__()
        self.initial_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim, elementwise_affine=False),
            nn.ReLU(),
            nn.Dropout(p=0.1)
        )
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(input_dim=hidden_dim, output_dim=hidden_dim) for _ in range(num_blocks)]
        )
        self.final_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.initial_layer(x)
        x = self.residual_blocks(x)
        x = self.final_layer(x)
        return x

class CustomVAEDecoder(nn.Module):
    def __init__(self, input_dim, vae_latent_dim, decoder_hidden_dim, output_dim, num_blocks=1):
        super(CustomVAEDecoder, self).__init__()
        self.mean_encoder = ResidualBlock(input_dim=input_dim, output_dim=vae_latent_dim)
        self.var_encoder = ResidualBlock(input_dim=input_dim, output_dim=vae_latent_dim)
        self.var_activation = torch.exp
        self.var_eps = 0.0001
        self.decoder = CustomDecoder(
            input_dim=vae_latent_dim,
            hidden_dim=decoder_hidden_dim,
            output_dim=output_dim,
            num_blocks=num_blocks
        )

    def forward(self, x, temperature=1.0):
        '''
            Returns:
                outputs:    output representations
                latents:    VAE latent representations
                dist:       a torch.dist object used to compute KL divergence
        '''
        # set_trace()
        mu = self.mean_encoder(x)
        var = self.var_encoder(x)
        var = self.var_activation(var) + self.var_eps
        dist = Normal(mu, temperature*(var.sqrt()))
        latents = dist.rsample()
        outputs = self.decoder(latents)
        # set_trace()
        return outputs, latents, dist

In [10]:
from transformers import GPT2Model, GPT2Config
class ProteinAngleFlowModel(nn.Module):
    def __init__(self, 
        input_size, 
        proj_hid_size,
        llm_embd_size,
        num_res_per_group=1,
        llm_name="gpt2",
        use_pretrained_weights=True,
        use_custom_gpt2_arch=False,
        llm_n_layer=1,
        llm_n_head=1,
        vae_latent_dim=128,
        decoder_hidden_dim=256,
    ):
        super().__init__()
        self.num_res_per_group = num_res_per_group
        self.proj_in = SimpleMLP(input_size=input_size, hidden_size=proj_hid_size, output_size=int(llm_embd_size/num_res_per_group))
        self._create_llm(llm_name, use_custom_gpt2_arch, use_pretrained_weights, llm_embd_size, llm_n_layer, llm_n_head)
        self.vae_decoder = CustomVAEDecoder(input_dim=llm_embd_size, vae_latent_dim=vae_latent_dim, decoder_hidden_dim=decoder_hidden_dim, output_dim=input_size*num_res_per_group)
    
    def _create_llm(self, llm_name, use_custom_gpt2_arch, use_pretrained_weights, llm_embd_size, llm_n_layer, llm_n_head):
        if llm_name == "gpt2": 
            print("Use GPT2 as LLM.")
            if not use_custom_gpt2_arch:
                print("Use GPT2 model with pretrained architecture!")
                if llm_embd_size != 768:
                    print(f"Override GPT2 dimension from {self._gpt_conf.n_embd} to 768!")
                if use_pretrained_weights:
                    self.llm_model = GPT2Model.from_pretrained('gpt2') 
                    print("Use GPT2 model with PRETRAINED weights!")
                else:
                    config = GPT2Config()  # Default GPT-2 configuration
                    self.llm_model = GPT2Model(config=config)
                    print("Use GPT2 model with RANDOM weights!")
            else:
                config = GPT2Config(
                    n_embd=llm_embd_size,
                    n_layer=llm_n_layer,
                    n_head=llm_n_head
                )
                self.llm_model = GPT2Model(config=config)
                print("Use CUSTOM GPT2 architecture!")
                print("Use GPT2 model with RANDOM weights!")
            self.llm_model.wte = None
        elif self.llm_name in ["pythia-160m", 'pythia-410m']:
            llm_name = 'EleutherAI/' + self._model_conf.llm_name
            print(f"Use {llm_name} as LLM.")
            if use_pretrained_weights:
                print(f"Use {llm_name} model with PRETRAINED weights!")
                self.llm_model = AutoModel.from_pretrained(llm_name)
            else:
                pythia_config = AutoConfig.from_pretrained(llm_name)
                self.llm_model = AutoModel.from_config(pythia_config)
                print(f"Use {llm_name} model with RANDOM weights!")

    def forward(self, protein_angles):
        residue_embeds = self.proj_in(protein_angles) # [b, t, s, d]
        _b, _t, _s, _d = residue_embeds.shape
        group_embeds = rearrange(residue_embeds, 'b t (s0 g) d -> b (t s0) (g d)', g = self.num_res_per_group) # [b, t*s/g, g*d], g: num_res_per_group
        llm_outputs = self.llm_model(inputs_embeds=group_embeds)
        llm_outputs_embeds = llm_outputs.last_hidden_state
        vae_outputs, vae_latents, vae_dist = self.vae_decoder(llm_outputs_embeds)
        outputs_embeds = rearrange(vae_outputs, 'b (t s0) (g d) -> b t (s0 g) d', s0=int(_s/self.num_res_per_group),g = self.num_res_per_group) # [b, t, s, d], g: num_res_per_group
        outputs_embeds = modulo_with_wrapped_range(outputs_embeds)
        # KL divergence loss
        z_cond_dist = vae_dist
        z_prior_dist = Normal(torch.zeros_like(vae_latents), torch.ones_like(vae_latents))
        # set_trace()
        kl_divergence_z = kl_divergence(
                        z_cond_dist,
                        z_prior_dist
                    )
        kl_divergence_z = rearrange(kl_divergence_z, 'b (t s0) (g d) -> b t (s0 g) d', s0=int(_s/self.num_res_per_group), g=self.num_res_per_group) #[b, t, s, d]
        kl_divergence_z = kl_divergence_z.sum(dim=-1)[:,:-1,:] # throw away the last time point
        return {
            'output_embeds': outputs_embeds,
            'kl_divergence': kl_divergence_z.mean()
        }
    
    def generate_next_tokens(self, protein_angles, temperature=1):
        assert not temperature < 0, "Temperature should be non-negative!"
        residue_embeds = self.proj_in(protein_angles) # [b, t, s, d]
        _b, _t, _s, _d = residue_embeds.shape
        group_embeds = rearrange(residue_embeds, 'b t (s0 g) d -> b (t s0) (g d)', g = self.num_res_per_group) # [b, t*s/g, g*d], g: num_res_per_group
        llm_outputs = self.llm_model(inputs_embeds=group_embeds)
        llm_outputs_embeds = llm_outputs.last_hidden_state
        vae_outputs, vae_latents, vae_dist = self.vae_decoder(llm_outputs_embeds, temperature=temperature)
        outputs_embeds = rearrange(vae_outputs, 'b (t s0) (g d) -> b t (s0 g) d', s0=int(_s/self.num_res_per_group),g = self.num_res_per_group) # [b, t, s, d], g: num_res_per_group
        outputs_embeds = modulo_with_wrapped_range(outputs_embeds)
        
        return outputs_embeds[:,-1,:,:].unsqueeze(1)

    def generate(self, input_ids, temperature=1, max_length=8):
        output_sequences = input_ids
        while output_sequences.shape[1] < max_length:
            # print(torch.cuda.memory_allocated())
            next_tokens = self.generate_next_tokens(output_sequences)
            output_sequences = torch.cat([output_sequences, next_tokens], dim=1)
        return output_sequences


In [11]:
protein_angle_model = ProteinAngleFlowModel(input_size=6, proj_hid_size=128, llm_embd_size=768).to("cuda")

Use GPT2 as LLM.
Use GPT2 model with pretrained architecture!


/home/sh2748/.conda/envs/foldingdiff/lib/python3.8/site-packages/transformers/modeling_utils.py:1331: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r

Use GPT2 model with PRETRAINED weights!


In [12]:
x0 = x0.unsqueeze(1).to(torch.float).to("cuda")

In [13]:
print(torch.cuda.memory_allocated())
aaa = protein_angle_model.generate(x0)
print(torch.cuda.memory_allocated())

358359552
41170448896


In [14]:
del aaa
print(torch.cuda.memory_allocated())

391913984


In [15]:
bbb=[]
print(torch.cuda.memory_allocated())
aaa = protein_angle_model.generate(x0)
# aaa_cpu = aaa.to("cpu")
# bbb.append(aaa_cpu)
print(torch.cuda.memory_allocated())
del aaa
print(torch.cuda.memory_allocated())

391913984
41171235328
391913984


In [27]:
bbb=[]
print(torch.cuda.memory_allocated())
aaa = protein_angle_model.generate(x0).cpu().detach().numpy()
print(torch.cuda.memory_allocated())
# aaa_cpu = aaa.clone().to("cpu")
# print(torch.cuda.memory_allocated())
torch.cuda.empty_cache()
# print(torch.cuda.memory_allocated())
bbb.append(aaa)
# print(torch.cuda.memory_allocated())
del aaa

print(torch.cuda.memory_allocated())
# del aaa_cpu
print(torch.cuda.memory_allocated())

391913984
391913984
391913984
391913984


In [23]:
del bbb[0]
print(torch.cuda.memory_allocated())

391913984


In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())

In [33]:
aaa.shape

torch.Size([8, 8, 128, 6])

In [36]:
from tqdm import tqdm
bbb = []
for i in tqdm(range(10)):
    # print(torch.cuda.memory_allocated())
    aaa = protein_angle_model.generate(x0)[:,-1,:,:].cpu().detach()
    bbb.append(aaa)


100%|██████████| 10/10 [00:02<00:00,  4.75it/s]


In [37]:
torch.cat(bbb, dim=0).shape

torch.Size([80, 128, 6])

: 

In [ ]:
output_sequences = x0.unsqueeze(1).to(torch.float).to("cuda")
while output_sequences.shape[1] < max_length:
    # print(output_sequences.shape[1])
    next_tokens = self.generate_next_tokens(output_sequences)
    # print(output_sequences.shape)
    # print(next_tokens.shape)
    output_sequences = torch.cat([output_sequences, next_tokens], dim=1)

In [ ]:
outputs = protein_angle_model(xt.to(torch.float))

In [ ]:
outputs.keys()

In [ ]:
xt.shape

## Training Loop

In [ ]:
lr = 0.00001
weight_decay = 0
beta = 0.3
num_epochs=100

In [ ]:
optimizer = torch.optim.Adam(protein_angle_model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
from tqdm import tqdm

protein_angle_model = protein_angle_model.to("cuda")
losses = []
for epoch in range(num_epochs):
    epoch_loss = 0
    for batch in tqdm(dataloader):
        batch = batch.to("cuda")
        x0 = torch.randn_like(batch)
        t, xt, mu_t = IFM.sample_conditional_flow(rearrange(x0, 'b s d -> b (s d)').to("cuda"), rearrange(batch, 'b s d -> b (s d)').to("cuda"), 8,  device = "cuda")
        xt = rearrange(xt, 'b t (s d) -> b t s d', s=x0.shape[1])
        xt = modulo_with_wrapped_range(xt)


        xt = xt.to(torch.float32)
    
        outputs = protein_angle_model(xt)
        shifted_pred_tokens = outputs["output_embeds"][:, :-1, ...]
        shifted_gt_tokens = xt[:,1:,...]

        mse_loss = F.mse_loss(shifted_pred_tokens, shifted_gt_tokens, reduction="none").sum(dim=-1).mean()
        kl_div = outputs['kl_divergence']
        loss = mse_loss + beta * kl_div
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        epoch_loss += loss.item()


       
    print("Epoch {}: loss: {}".format(epoch+1, epoch_loss))
        